<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Cargado de datos

In [41]:
dist_cepii = pd.read_stata('https://github.com/vicentcamison/ICC-Data-Hackathon-2021/blob/main/Refugee%20Crisis/dist_cepii.dta?raw=true')
fatalities = pd.read_csv('https://raw.githubusercontent.com/vicentcamison/ICC-Data-Hackathon-2021/main/Refugee%20Crisis/fatalities.csv')
forcibly_displaced = pd.read_csv('https://raw.githubusercontent.com/vicentcamison/ICC-Data-Hackathon-2021/main/Refugee%20Crisis/forciby_displaced.csv', encoding='ISO-8859-1')
geo_cepii = pd.read_stata('https://github.com/vicentcamison/ICC-Data-Hackathon-2021/blob/main/Refugee%20Crisis/geo_cepii.dta?raw=true')
terrain_info = pd.read_csv('https://raw.githubusercontent.com/vicentcamison/ICC-Data-Hackathon-2021/main/Refugee%20Crisis/terrain_info.csv', encoding='ISO-8859-1')

In [16]:
forcibly_displaced

,Unnamed: 0,Year,CountryOriginCode,CountryAsylumCode,CountryOriginName,CountryAsylumName,REF,IDP,ASY
0,1,1951,UKN,AUS,NaN,Australia,180000.0,0.0,0
1,2,1951,UKN,AUT,NaN,Austria,282000.0,0.0,0
2,3,1951,UKN,BEL,NaN,Belgium,55000.0,0.0,0
3,4,1951,UKN,CAN,NaN,Canada,168511.0,0.0,0
4,5,1951,UKN,DNK,NaN,Denmark,2000.0,0.0,0
...,...,...,...,...,...,...,...,...,...
103297,103298,2019,ZWE,SWE,Zimbabwe,Sweden,19.0,0.0,5
103298,103299,2019,ZWE,CHE,Zimbabwe,Switzerland,13.0,0.0,5
103299,103300,2019,ZWE,THA,Zimbabwe,Thailand,0.0,0.0,9
103300,103301,2019,ZWE,USA,Zimbabwe,United States of America,825.0,0.0,867


In [91]:
# Hay muchos NAN, hay que limpiarlos
forcibly_displaced_grouped = forcibly_displaced.dropna()
forcibly_displaced_grouped = forcibly_displaced_grouped.drop(['CountryOriginName', 'CountryAsylumName', 'CountryAsylumCode'], axis=1)
forcibly_displaced_grouped = forcibly_displaced_grouped.groupby(['CountryOriginCode', 'Year'])
forcibly_displaced_grouped = forcibly_displaced_grouped.sum()
forcibly_displaced_grouped = forcibly_displaced_grouped.drop(['Unnamed: 0'], axis=1)

In [94]:
forcibly_displaced_grouped

REF       IDP    ASY
CountryOriginCode Year                            
AFG               1980  1434921.0       0.0      0
                  1981  3879984.0       0.0      0
                  1982  4488214.0       0.0      0
                  1983  4712735.0       0.0      0
                  1984  4417651.0       0.0      0
...                           ...       ...    ...
ZWE               2015    21332.0       0.0  57376
                  2016    18141.0       0.0  43191
                  2017    17420.0       0.0  21760
                  2018    15618.0       0.0  17121
                  2019    10603.0  270000.0   3352

[6266 rows x 3 columns]

In [151]:
countries = forcibly_displaced.loc[:, 'CountryOriginCode'].unique()
countries

array(['UKN', 'AGO', 'GHA', 'RWA', 'GNB', 'SDN', 'ALB', 'ARM', 'CHN',
       'COD', 'MOZ', 'RUS', 'VNM', nan, 'BDI', 'COG', 'GRC', 'HUN', 'CPV',
       'MWI', 'NAM', 'ZAF', 'ETH', 'IRQ', 'NGA', 'SYR', 'ZMB', 'BGR',
       'CUB', 'SRB', 'BOL', 'TCD', 'CZE', 'POL', 'ROU', 'ESP', 'BRA',
       'KHM', 'GIN', 'IND', 'LSO', 'MMR', 'TUR', 'UGA', 'CHL', 'GNQ',
       'HTI', 'PAK', 'TZA', 'URY', 'KEN', 'LAO', 'SOM', 'ESH', 'ZWE',
       'BEN', 'PSE', 'CYP', 'PHL', 'EGY', 'DOM', 'IDN', 'NIC', 'YEM',
       'AFG', 'ERI', 'ARG', 'GTM', 'IRN', 'LBY', 'SLV', 'STA', 'DZA',
       'COL', 'CAF', 'LBN', 'TUN', 'BFA', 'JOR', 'LKA', 'STP', 'TGO',
       'HND', 'BGD', 'GRD', 'ISR', 'LBR', 'MAR', 'PAN', 'PER', 'SEN',
       'SYC', 'SGP', 'SLE', 'BWA', 'CMR', 'COM', 'MRT', 'SUR', 'BHR',
       'DJI', 'ECU', 'GMB', 'KWT', 'LTU', 'PRY', 'THA', 'BTN', 'CAN',
       'DEU', 'CIV', 'KOR', 'MLI', 'MYS', 'USA', 'AZE', 'BIH', 'CRI',
       'EST', 'GUF', 'FJI', 'GAB', 'HRV', 'LVA', 'NER', 'TJK', 'UKR',
       'VEN', '

In [216]:
len(countries)

211

## TAREA 1: Juntar la información de terrain_info y geo_cepii, y aunarlos por el país

de aquí obenemos los dataframes **geo_cepii_clean** y **terrain_info_clean**. Únicamente faltaría normalizarlos y aunarlos para poder usarlos como fuente de información adicional en unas capas densas

### geo_cepii

In [120]:
geo_cepii.head(1)

#El tratamiento que se va a hacer es el siguiente:
# La limpieza de los datos va a estar determinada por la relativa simplicidad de ser capaz de incorporarlos a un modelo relativamente sencillo,
# como complemento al resto de variables.
# Cosas que se ven a simple vista cuyo impacto va a ser relativamente pequeño (y que requieran de un tratamiento mayor para llevar a cabo satisfactoriamente), serán eliminados
# [iso2], [cnum], [country], [pays] : todos hacen referencia a los países. Esta referencia pasará a ser el índice en el megadataframe (nos quedamos pr ahora con [iso3] que es el común en el resto de df)
# [city_en], [city_fr] ofrece información con la que no se puede hacer nada con estos datos (el nombre de la capital)
# [citynum] habla de que su uso es interesante en secciones posteriores, pero no hay secciones posteriores
# [langoff2], [langoff3] poco relevantes: hay que hilar muy fino para encontrar el uso, y hay que lidiar con el problema de los NaN PERO PUEDE SER UN PUNTO INTERESANTE AL QUE VOLVER
# [TODAS LAS SIGUIENTES] eliminadas porque tirnen muchos NaN y la info es de poca importancia

#TRATAMIENTO ADICIONAL
# [langoff_1] y [lang20_1] se van a eliminar en un primer pase porque requieren de algún tratamiento especial para ser prácticas (dummificarlo quizás es demasiado aparatoso para el modelo, y embedding lleva tiempo)
# sin embargo PUEDE SER UN PUNTO INTERESANTE AL QUE VOLVER

,iso2,iso3,cnum,country,pays,area,dis_int,landlocked,continent,city_en,city_fr,lat,lon,cap,maincity,citynum,langoff_1,langoff_2,langoff_3,lang20_1,lang20_2,lang20_3,lang20_4,lang9_1,lang9_2,lang9_3,lang9_4,colonizer1,colonizer2,colonizer3,colonizer4,short_colonizer1,short_colonizer2,short_colonizer3
0,AD,AND,20,Andorra,Andorre,453,8.005398,0.0,Europe,Andorra la Vella,Andorre-la-Vieille,42.5,1.5,1,1,7.0,Catalan,Spanish,,Catalan,Spanish,,,,,,,,,,,,,


In [123]:
geo_cepii_clean = geo_cepii.drop(['iso2', 'cnum', 'country', 'pays', 'city_en', 'city_fr', 'citynum', 'langoff_2', 'langoff_3', 'lang20_1', 'lang20_2', 'lang20_3', 'lang20_4', 'lang9_1', 'lang9_2', 'lang9_3', 'lang9_4', 'colonizer1', 'colonizer2', 'colonizer3', 'colonizer4', 'short_colonizer1', 'short_colonizer2', 'short_colonizer3'], axis=1)
geo_cepii_clean.head(5)

,iso3,area,dis_int,landlocked,continent,lat,lon,cap,maincity,langoff_1
0,AND,453,8.005398,0.0,Europe,42.500000,1.500000,1,1,Catalan
1,ARE,83657,108.788994,0.0,Asia,24.466667,54.416668,1,1,Arabic
2,AFG,652225,303.761353,1.0,Asia,34.516666,69.199997,1,1,Persian
3,ATG,442,7.907605,0.0,America,17.133333,-61.833332,1,1,English
4,AIA,102,3.798690,0.0,America,18.216667,-63.066666,1,1,English


In [124]:
geo_cepii_clean.isna().sum().sum()
# Con esta cantidad de datos en el df, también nos hemos cargado todos los NaN pero, ¿quedan elementos vacíos?

0

In [128]:
np.logical_and((geo_cepii_clean == '').values, (geo_cepii_clean == None).values).sum().sum()
# Tampoco hay elementos vacíos: el dataframe está definido al 100%

0

In [133]:
#Trabajemos ahora por preparar los datos para funcionar:
# ¿Cuántas lenguas distintas hay?
geo_cepii_clean.langoff_1.unique().shape[0]
# Hay 69 lenguas distintas: demasiadas para dummificar. Quitaremos también esa columna
geo_cepii_clean.drop('langoff_1', axis=1, inplace=True)

#Hacemos los dummies del continente
geo_cepii_clean = pd.get_dummies(geo_cepii_clean, columns=['continent'], drop_first=True)

AttributeError: ignored

In [138]:
geo_cepii_clean

,iso3,area,dis_int,landlocked,lat,lon,cap,maincity,continent_America,continent_Asia,continent_Europe,continent_No,continent_Pacific
0,AND,453,8.005398,0.0,42.500000,1.500000,1,1,0,0,1,0,0
1,ARE,83657,108.788994,0.0,24.466667,54.416668,1,1,0,1,0,0,0
2,AFG,652225,303.761353,1.0,34.516666,69.199997,1,1,0,1,0,0,0
3,ATG,442,7.907605,0.0,17.133333,-61.833332,1,1,1,0,0,0,0
4,AIA,102,3.798690,0.0,18.216667,-63.066666,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,ZAF,1219912,415.430206,0.0,-33.933334,18.466667,2,1,0,0,0,0,0
234,ZAF,1219912,415.430206,0.0,-25.730000,28.219999,1,0,0,0,0,0,0
235,ZMB,752614,326.302155,1.0,-15.466666,28.266666,1,1,0,0,0,0,0
236,ZAR,2345410,576.027466,0.0,-4.300000,15.300000,1,1,0,0,0,0,0


In [209]:
# los continentes aparecen ya en el siguiente dataset
geo_cepii_clean.drop(['continent_America', 'continent_Asia', 'continent_Europe', 'continent_No', 'continent_Pacific'], axis=1, inplace=True)

In [214]:
#esta es la lista de países con la que nos vamos a quedar
countries = forcibly_displaced.loc[:, 'CountryOriginCode'].unique()

#mask = terrain_info.isocode.apply(lambda x: any(item for item in countries if item in x))
#terrain_info_clean = terrain_info[mask]
mask = geo_cepii_clean.iso3.isin(countries)
geo_cepii_clean_clean = geo_cepii_clean[mask]

In [215]:
geo_cepii_clean
#WAIT, hay columnas del mismo país! (será para diferentes ciudades grandes) eso no lo he tenido en cuenta!!

,iso3,area,dis_int,landlocked,lat,lon,cap,maincity
0,AND,453,8.005398,0.0,42.500000,1.500000,1,1
1,ARE,83657,108.788994,0.0,24.466667,54.416668,1,1
2,AFG,652225,303.761353,1.0,34.516666,69.199997,1,1
3,ATG,442,7.907605,0.0,17.133333,-61.833332,1,1
4,AIA,102,3.798690,0.0,18.216667,-63.066666,1,1
...,...,...,...,...,...,...,...,...
233,ZAF,1219912,415.430206,0.0,-33.933334,18.466667,2,1
234,ZAF,1219912,415.430206,0.0,-25.730000,28.219999,1,0
235,ZMB,752614,326.302155,1.0,-15.466666,28.266666,1,1
236,ZAR,2345410,576.027466,0.0,-4.300000,15.300000,1,1


### terrain_info

In [148]:
terrain_info

# En este primer pase, sólamente nos quedaremos con las columnas que se correspondan con alguna de los países de salida de los refugiados, detallado en el dataset forcibly_displaced

#El tratamiento que se va a hacer es el siguiente:
# La limpieza de los datos va a estar determinada por la relativa simplicidad de ser capaz de incorporarlos a un modelo relativamente sencillo,
# como complemento al resto de variables.
# Cosas que se ven a simple vista cuyo impacto va a ser relativamente pequeño (y que requieran de un tratamiento mayor para llevar a cabo satisfactoriamente), serán eliminados
# [isonum], [country] info del país que ya tenemos en [isocode]
# [rgdppc_2000], [rgdppc_1950_m], [rgdppc_1975_m], [rgdppc_2000_m], [rgdppc_1950_2000_m], [q_rule_law] a pesar de que parece info interesante, hay demasiados NaNs como para gestionarla correctamente
# [dist_slave....] demasiados NaNs y demasiado específica quizás. Para saber info de esclavos, tenemos [slave_exports]
# [cont_africa] todos los países de áfrica presentan información adicional unas columnas más adelante sobre en qué lugar de áfrica concreto se encuentran.
# [pop 1400] ni sé lo que es ni me importa
# [european_descent] ni sé lo que es ni me importa

#TRATAMIENTO DE NaNs
# [land_area] los países que tienen NaN es porque son pequeños: cambiamos los NaNs por 0
# [soil] países pequeños: se sustituye también por 0
# [legor_...] hay 23 NaNs, y creo que son todos en los mismos países: vamos a sustituírlos todos por 0: de esta forma, le estamos diciendo a la red que ese país no tiene ninguno de los regímenes mencionados
# En resumen, los NaNs de estas 3 columnas recibirán el mismo tratamiento: serán reemplazadas por 0

#DESPUÉS DE UN PREPROCESADO
# [rgdppc...] todas las columnas asociadas al PIB desaparecen después de añadir la columna [gdc]

,isocode,isonum,country,rugged,rugged_popw,rugged_slope,rugged_lsd,rugged_pc,land_area,lat,lon,soil,desert,tropical,dist_coast,near_coast,gemstones,rgdppc_2000,rgdppc_1950_m,rgdppc_1975_m,rgdppc_2000_m,rgdppc_1950_2000_m,q_rule_law,cont_africa,cont_asia,cont_europe,cont_oceania,cont_north_america,cont_south_america,legor_gbr,legor_fra,legor_soc,legor_deu,legor_sca,colony_esp,colony_gbr,colony_fra,colony_prt,colony_oeu,africa_region_n,africa_region_s,africa_region_w,africa_region_e,africa_region_c,slave_exports,dist_slavemkt_atlantic,dist_slavemkt_indian,dist_slavemkt_saharan,dist_slavemkt_redsea,pop_1400,european_descent
0,ABW,533,Aruba,0.462,0.380,1.226,0.144,0.000,18.0,12.508,-69.970,21.324,0.000,100.000,0.001,100.0000,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0.0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,NaN,614.0,NaN
1,AFG,4,Afghanistan,2.518,1.469,7.414,0.720,39.004,65209.0,33.833,66.026,27.849,4.356,0.000,0.922,0.0000,0,NaN,644.756,720.633,565.231,679.791,-1.687,0,1,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,NaN,1870829.0,0.0
2,AGO,24,Angola,0.858,0.714,2.274,0.228,4.906,124670.0,-12.299,17.551,26.676,0.425,44.346,0.428,13.1587,47756,1794.729,1051.822,1073.036,765.215,1106.763,-1.567,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,1,3610000.000,5.669,6.981,4.926,3.872,1223208.0,2.0
3,AIA,660,Anguilla,0.013,0.010,0.026,0.006,0.000,9.0,18.231,-63.064,100.000,0.000,100.000,0.000,100.0000,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,NaN,NaN,NaN
4,ALB,8,Albania,3.427,1.597,10.451,1.006,62.133,2740.0,41.143,20.070,68.088,0.000,0.000,0.048,94.6919,0,3703.113,1001.339,2289.472,2741.420,1931.784,-0.820,0,0,1,0,0,0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,NaN,200000.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,WSM,882,Samoa,1.677,0.968,4.644,0.549,22.244,283.0,-13.738,-172.168,100.000,0.000,100.000,0.003,100.0000,0,4860.269,NaN,NaN,NaN,NaN,0.360,0,0,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,NaN,33072.0,NaN
230,YEM,887,Yemen,2.323,2.197,6.155,0.591,33.060,52797.0,15.863,47.580,0.000,10.750,0.000,0.148,38.1163,0,787.749,911.231,1783.548,2619.284,1703.219,-1.127,0,1,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,NaN,2250000.0,0.0
231,ZAF,710,South Africa,1.761,1.318,4.942,0.450,23.957,121447.0,-29.004,25.094,27.647,4.386,0.179,0.306,18.9404,144785,9419.122,2534.946,4270.851,3978.219,3667.399,0.207,1,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,1,0,0,0,2030.663,6.766,3.457,6.584,4.895,486527.0,18.0
232,ZMB,894,Zambia,0.533,0.506,1.422,0.145,2.455,74339.0,-13.458,27.794,23.114,0.000,17.132,0.900,0.0000,0,774.010,660.910,1040.744,645.375,863.642,-0.587,1,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,1,0,0,0,27957.102,9.027,2.389,4.849,3.253,348220.0,1.0


In [157]:
#esta es la lista de países con la que nos vamos a quedar
countries = forcibly_displaced.loc[:, 'CountryOriginCode'].unique()
countries

#mask = terrain_info.isocode.apply(lambda x: any(item for item in countries if item in x))
#terrain_info_clean = terrain_info[mask]
mask = terrain_info.isocode.isin(countries)
terrain_info_clean = terrain_info[mask]

In [189]:
#para las entries de los dineros que no son nulos, nos quedamos con el valor existente entre los dos que no sea nulo, o con la media si existen ambos
# los NaNs los dejamos estar por ahora
for index, row in terrain_info_clean.iterrows():
  if not pd.isna(terrain_info_clean.loc[index, 'rgdppc_2000']) and not pd.isna(terrain_info_clean.loc[index, 'rgdppc_2000_m']):
    terrain_info_clean.loc[index, 'gdp'] = (terrain_info_clean.loc[index, 'rgdppc_2000'] + terrain_info_clean.loc[index, 'rgdppc_2000_m']) / 2
  elif pd.isna(terrain_info_clean.loc[index, 'rgdppc_2000']) and not pd.isna(terrain_info_clean.loc[index, 'rgdppc_2000_m']):
    terrain_info_clean.loc[index, 'gdp'] = terrain_info_clean.loc[index, 'rgdppc_2000_m']
  elif not pd.isna(terrain_info_clean.loc[index, 'rgdppc_2000']) and pd.isna(terrain_info_clean.loc[index, 'rgdppc_2000_m']):
    terrain_info_clean.loc[index, 'gdp'] = terrain_info_clean.loc[index, 'rgdppc_2000']
  else:
    terrain_info_clean.loc[index, 'gdp'] = float("nan")

#calculamos las medias para cada una de las regiones: asignaremos estos valores a los NaNs de cada una de las regiones:
location = ['africa_region_n', 'africa_region_s', 'africa_region_w', 'africa_region_e', 'africa_region_c', 'cont_asia', 'cont_europe', 'cont_oceania', 'cont_north_america', 'cont_south_america']
mean = np.zeros(shape=len(location))
for i in range(len(mean)):
  mean[i] = np.nanmean(terrain_info_clean[terrain_info_clean.loc[:, location[i]] == 1].loc[:, 'gdp'])

#una vez ya tenemos las medias, las colocamos en este diccionario para asignar las medias:
# [africa_region_n, africa_region_s, africa_region_w, africa_region_e, africa_region_c, cont_asia, cont_europe, cont_oceania, cont_north_america, cont_south_america]
continent_dic = {tuple([1,0,0,0,0,0,0,0,0,0]): mean[0],
                 tuple([0,1,0,0,0,0,0,0,0,0]): mean[1],
                 tuple([0,0,1,0,0,0,0,0,0,0]): mean[2],
                 tuple([0,0,0,1,0,0,0,0,0,0]): mean[3],
                 tuple([0,0,0,0,1,0,0,0,0,0]): mean[4],
                 tuple([0,0,0,0,0,1,0,0,0,0]): mean[5],
                 tuple([0,0,0,0,0,0,1,0,0,0]): mean[6],
                 tuple([0,0,0,0,0,0,0,1,0,0]): mean[7],
                 tuple([0,0,0,0,0,0,0,0,1,0]): mean[8],
                 tuple([0,0,0,0,0,0,0,0,0,1]): mean[9]}

#volvemos a pasar por los gdp para colocar los valores obtenidos de las medias:
for index, row in terrain_info_clean.iterrows():
  if pd.isna(terrain_info_clean.loc[index, 'rgdppc_2000']) and pd.isna(terrain_info_clean.loc[index, 'rgdppc_2000_m']):
    dic_entry = tuple([terrain_info_clean.loc[index, 'africa_region_n'], terrain_info_clean.loc[index, 'africa_region_s'], terrain_info_clean.loc[index, 'africa_region_w'], terrain_info_clean.loc[index, 'africa_region_e'],
                                                              terrain_info_clean.loc[index, 'africa_region_c'], terrain_info_clean.loc[index, 'cont_asia'], terrain_info_clean.loc[index, 'cont_europe'], terrain_info_clean.loc[index, 'cont_oceania'],
                                                              terrain_info_clean.loc[index, 'cont_north_america'], terrain_info_clean.loc[index, 'cont_south_america']])

    terrain_info_clean.loc[index, 'gdp'] = continent_dic[dic_entry]


(0, 0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 0, 1, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 1, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0)
(1, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1)
(0, 0, 0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 1, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 0, 1, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 0, 0, 0, 1, 0, 0, 0)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [210]:
#eliminamos las columnas redundantes y llenamos los NaN de los elementos restantes con 0 (ya se ha argumentado antes el porqué de esta decisión)

terrain_info_clean.drop(['isonum', 'country', 'rgdppc_2000', 'rgdppc_1950_m', 'rgdppc_1975_m', 'rgdppc_2000_m', 'rgdppc_1950_2000_m', 'q_rule_law', 'dist_slavemkt_atlantic',
                         'dist_slavemkt_indian', 'dist_slavemkt_saharan', 'dist_slavemkt_redsea', 'pop_1400', 'european_descent'], axis=1, inplace=True)
terrain_info_clean.fillna(0, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,isocode,rugged,rugged_popw,rugged_slope,rugged_lsd,rugged_pc,land_area,lat,lon,soil,desert,tropical,dist_coast,near_coast,gemstones,cont_africa,cont_asia,cont_europe,cont_oceania,cont_north_america,cont_south_america,legor_gbr,legor_fra,legor_soc,legor_deu,legor_sca,colony_esp,colony_gbr,colony_fra,colony_prt,colony_oeu,africa_region_n,africa_region_s,africa_region_w,africa_region_e,africa_region_c,slave_exports,gdp
1,AFG,2.518,1.469,7.414,0.720,39.004,65209.0,33.833,66.026,27.849,4.356,0.000,0.922,0.0000,0,0,1,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.000,565.231000
2,AGO,0.858,0.714,2.274,0.228,4.906,124670.0,-12.299,17.551,26.676,0.425,44.346,0.428,13.1587,47756,1,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,1,3610000.000,1279.972000
4,ALB,3.427,1.597,10.451,1.006,62.133,2740.0,41.143,20.070,68.088,0.000,0.000,0.048,94.6919,0,0,0,1,0,0,0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.000,3222.266500
5,AND,5.717,6.722,17.774,1.616,99.064,47.0,42.551,1.576,0.000,0.000,0.000,0.134,0.0000,0,0,0,1,0,0,0,0.0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.000,16315.372473
7,ARE,0.769,0.316,2.112,0.191,6.142,8360.0,23.913,54.331,0.000,77.280,0.000,0.065,75.7464,0,0,1,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0.000,19086.171500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,WSM,1.677,0.968,4.644,0.549,22.244,283.0,-13.738,-172.168,100.000,0.000,100.000,0.003,100.0000,0,0,0,0,1,0,0,1.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0.000,4860.269000
230,YEM,2.323,2.197,6.155,0.591,33.060,52797.0,15.863,47.580,0.000,10.750,0.000,0.148,38.1163,0,0,1,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0.000,1703.516500
231,ZAF,1.761,1.318,4.942,0.450,23.957,121447.0,-29.004,25.094,27.647,4.386,0.179,0.306,18.9404,144785,1,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,1,0,0,0,2030.663,6698.670500
232,ZMB,0.533,0.506,1.422,0.145,2.455,74339.0,-13.458,27.794,23.114,0.000,17.132,0.900,0.0000,0,1,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,1,0,0,0,27957.102,709.692500
